In [21]:
import math
from json import JSONEncoder

class depotEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__

depot_name = "e-Depot Tan Binh"
outfile = "./etdv2.json"

In [22]:
class Point:
	def __init__(self,x,y) -> None:
		self.x = x
		self.y = y
	
	def translate(self, dx=0, dy=0):
		self.x += dx
		self.y += dy

	def scale(self, cox=1, coy=1):
		self.x *= cox
		self.y *= coy

class Shape:
	def __init__(self, seq, vis=1) -> None:
		self.seq = seq
		self.length = len(seq)
		self.visible = vis

	def scale(self,cox=1, coy=1):
		for i in range(self.length):
			self.seq[i].scale(cox,coy)
	
	def translate(self,dx=0, dy=0):
		for i in range(self.length):
			self.seq[i].translate(dx,dy)

class Container: 
	def __init__(self, contID, contSize, HangTau, Block, Bay, Row, Tier):
		self.ContID = contID
		self.ContTypeSizeID = contSize
		self.HangTauID = HangTau
		self.Block = Block
		self.Bay = Bay
		self.Row = Row
		self.Tier = Tier
		self.angle = 0
		self.x=0
		self.y=0
		self.z=0

class Text: 
	def __init__(self, content, x,y,size=8, angle=0) -> None:
		self.content = content
		self.position = Point(x,y)
		self.size = size
		self.angle = angle


In [23]:
# read text
def read_text(path):
    result = []
    fi = open(path, "r")
    str = fi.read()
    layout_array = []
    a = 0
    p = str.find("<text",a)
    while p>0:
        x = []
        y = []
        b = str.find("/text>",p)
        k = str[p+27:b]
        # print(k)
        text = k[k.find('">')+2:k.find("</")]
        
        k = k[0:k.find("rotate")]
        obr = k.find("(")
        cbr = k.find(")")
        translate = k[obr+1:cbr]
        # print(translate)

        sp = translate.find(" ")
        x = translate[0:sp]
        y = translate[sp+1: len(translate)]

        result.append(Text(text, int(x),int(y)))

        # print("x", x, "y", y)
        a = b
        p = str.find("<text",a)
    return result

def svg2layout(path):
    file = open(path, "r")
    str = file.read()
    layout_array = []
    a = 0
    p = str.find("<polygon",a)
    while p>0:
        x = []
        y = []
        b = str.find("/>",p)
        div = str[p+27:b-1] 
        # print("shape:", div)
        chr_id=0
        type = 0
        sp = div.find(" ",chr_id)
        while sp>0:
            if type:
                # print("y:",div[chr_id:sp])
                y.append(int(div[chr_id:sp]))
            else:
                # print("x:",div[chr_id:sp])
                x.append(int(div[chr_id:sp]))

            type = 1-type
            chr_id = sp
            sp = div.find(" ", sp+1)
        # print("y:",div[chr_id:len(div)])
        y.append(int(div[chr_id:len(div)]))

        a = b
        p = str.find("<polygon",a)
        # print(x)
        # print(y)
        seq = []
        for i in range(len(x)):
            seq.append(Point(x[i],y[i]))
        layout_array.append(Shape(seq))
    return layout_array



In [24]:
class Area:
    def __init__(self, name, x, y, offsetX=0, offsetY=0, angle=0, x_flip=False, y_flip=False, shapeID=None, num_of_bay=15, num_of_row=6, num_of_tier=6) -> None:
        self.name = name
        self.num_of_bay = num_of_bay
        self.num_of_row = num_of_row
        self.num_of_tier = num_of_tier
        self.x_coor = x
        self.y_coor = y
        self.shapeID = shapeID
        self.angle = angle
        self.offset = Point(offsetX, offsetY)
        self.x_flip = x_flip
        self.y_flip = y_flip


class Layout:
    def __init__(self,name,svg_path, text=[]) -> None:
        self.name = name
        self.shape = svg2layout(svg_path)
        self.text = read_text(svg_path)

    def scale(self,cox=1, coy=1):
        for i in range(len(self.shape)):
            self.shape[i].scale(cox,coy)
        for i in range(len(self.text)):
            self.text[i].position.scale(cox, coy)
    
    def translate(self,dx=0, dy=0):
        for i in range(len(self.shape)):
            self.shape[i].translate(dx,dy)
        for i in range(len(self.text)):
            self.text[i].position.translate(dx, dy)

class Ground:
    def __init__(self, shapeID, id1, offsetX=0, offsetY=0, offsetZ=0, angle=0):
        self.shapeID = shapeID
        self.id1 = id1
        self.offsetX = offsetX
        self.offsetY = offsetY
        self.offsetZ = offsetZ
        self.angle = angle
    
class House:
    def __init__(self, type, name, shape, id1=0, id2=2, height=100, angle=0, offsetX=0, offsetY=0) -> None:
        self.type = type
        self.name = name
        self.shape = shape
        self.id1 = id1
        self.id2 = id2
        self.height = height
        self.angle = angle
        self.offsetX = offsetX
        self.offsetY = offsetY

class Depot:
    def __init__(self, name, layout, ground=[], slope=[], area=[], house=[], container_scale=1, num_of_ground_level=1, offset=Point(0,0), scale=1) -> None:        
        self.name = name
        self.layout = layout
        self.contWidth = 8 * container_scale
        self.contHeight = 8.5 * container_scale
        self.contLength = 20 * container_scale
        self.contHalfLength = 10 * container_scale
        self.contGap = math.floor(self.contLength*0.06)
        self.fontSize = 3*container_scale   
        self.Area = area
        self.house = house
        self.ground=ground
        self.offset = offset
        self.scale = scale


In [25]:
(46/360)*(2*math.pi);

In [26]:
etd_layout = Layout("General","./etdv1.svg");
etd_layout.scale(cox=3, coy=3);
etd_layout.translate(-400, -1700);

# etd_layout.shape[0].visible = 0.2
# etd_layout.shape[1].visible = 0.2
# etd_layout.shape[2].visible = 0.2
# etd_layout.shape[3].visible = 0.2
# etd_layout.shape[4].visible = 0
etd_layout.shape[9].visible = 0


ground0 = Ground(0,4, offsetX=etd_layout.shape[0].seq[1].x, offsetY=etd_layout.shape[4].seq[1].y, offsetZ=0, angle=1.127010721276371)
ground1 = Ground(1,3, offsetX=etd_layout.shape[1].seq[2].x, offsetY=etd_layout.shape[2].seq[2].y, offsetZ=0, angle=-0.8158514559173915)

# print(etd_layout.shape[0].seq[8].x)
# add area here
# Area_A = Area("A", etd_layout.shape[2].seq[6].x, etd_layout.shape[2].seq[6].y, shapeID=2, x_flip=True, num_of_row=7, num_of_bay=13)
# Area_B = Area("B", etd_layout.shape[3].seq[4].x, etd_layout.shape[3].seq[4].y, shapeID=3, x_flip=True, num_of_bay=13, num_of_row=11)
# Area_C = Area("C", etd_layout.shape[4].seq[0].x, etd_layout.shape[4].seq[0].y, shapeID=4, angle=0.02583087292951608, offsetX=-42, offsetY=16, num_of_row=5, num_of_bay=4)
# Area_E = Area("D", etd_layout.shape[6].seq[1].x, etd_layout.shape[6].seq[1].y, shapeID=6, angle=0.05235987755982988, offsetX=52, offsetY=5, num_of_row=14,num_of_bay=9)
# Area_SC = Area("SC", etd_layout.shape[5].seq[1].x, etd_layout.shape[5].seq[1].y, shapeID=5, angle=0.05235987755982988, num_of_bay=7, num_of_row=3)
# Area_F = Area("F", etd_layout.shape[7].seq[8].x, etd_layout.shape[7].seq[8].y, shapeID=7, angle=0.05235987755982988, offsetX=80, offsetY=8, num_of_row=9, num_of_bay=10)


# add warehouse here
wh1 = House(0, "Kho 1", etd_layout.shape[2],0,2,200,0.05235987755982988)
wh2 = House(0, "kho 2", etd_layout.shape[3],2,0, 200, 0.05235987755982988)
wh3 = House(0, "Nha Kho", etd_layout.shape[4], 3,1,100,0.05235987755982988)
h4 = House(0, "ruacont", etd_layout.shape[5], 3,1,100,0.05235987755982988)
h5 = House(0, "ruacont", etd_layout.shape[6], 3,1,100,0.05235987755982988)
h6 = House(0, "ruacont", etd_layout.shape[7], 3,1,100,0.05235987755982988)
h7 = House(0, "ruacont", etd_layout.shape[8], 3,1,100,0.05235987755982988)
h8 = House(0, "ruacont", etd_layout.shape[9], 3,1,100,0.05235987755982988)

# ruaCont = House(2, "Rua container", etd_layout.shape[4],3,1,20, offsetX=34, offsetY=-74)
std = Depot("edepot Tan Binh", etd_layout, ground=[ground0, ground1], house=[wh1, wh2, wh3, h4,h5,h6,h7,h8], container_scale=2.1, scale=0.4)
# etd = Depot("etd", etd_layout, ground=[ground1, ground2], offset=Point(-450,300), house=[wh1, wh2, wh3, ruaCont],container_scale=4.25, scale=0.4)

In [27]:
jn = (depotEncoder().encode(std))
ofile = open(outfile, "w")
ofile.write(jn)
ofile.close()